
## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

The French mathematician Emile Borel once mused that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of **Recurrent Neural Networks and LSTMs**!

Our goal in this projectis to build a **Shakespeare Sonnet Generator**.<br>
Given a prompt of a few words as input, its task is to generate follow-on text that reads like a Shakespeare Sonnet!<br>

To build our Sonnet Generator we will use a type of model called a **sequence model**. Given a short sequence, a sequence  model predicts the **most likely next item in the sequence**. Sequence models are astonishingly versatile and powerful, because the **sequence** we want to predict can be quite general! It can be composed of **words**, or of **characters**, or of **musical notes**, or of data points in a **time series** such as EKG voltages, or stock prices, or even a sequence of **DNA nucleotides**! 

We will train our model on the entire corpus of Shakespeare's Sonnets, and the model will learn from that data the most likely patterns of characters.

In [ ]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# import a custom text data preparation class
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/data_cleaning_toolkit_class.py
from data_cleaning_toolkit_class import data_cleaning_toolkit

--2023-01-05 02:23:52--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/data_cleaning_toolkit_class.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6666 (6.5K) [text/plain]
Saving to: ‘data_cleaning_toolkit_class.py’

data_cleaning_toolk 100%[===================>]   6.51K  --.-KB/s    in 0s      

2023-01-05 02:23:52 (65.8 MB/s) - ‘data_cleaning_toolkit_class.py’ saved [6666/6666]



### Use `request` to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) to learn how to download the Shakespeare Sonnets from the Gutenberg website. 


In [ ]:
# download all of Shakespeare's Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use requests and the url to download all of the sonnets - save the result to `data`
data = requests.get(url_shakespeare_sonnets)


In [ ]:
dir(data)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [ ]:
# extract the downloaded text from the requests object and save it to `raw_text_data`
# hint: take a look at the attributes of `data`
# YOUR CODE HERE
raw_text_data = data.text
#raise NotImplementedError()
raw_text_data

'\ufeffThe Project Gutenberg eBook of The Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: The Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease Date: September, 1997 [eBook #1041]\r\n[Most recently updated: December 3, 2022]\r\n\r\nLanguage: English\r\n\r\n\r\nProduced by:  the Project Gutenberg Shakespeare Team\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE SONNETS ***\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\nI\r\n\r\nFrom fairest creatures we desire increase,\r\nThat thereby 

In [ ]:
# check the data type of `raw_text_data`
assert(type(raw_text_data)==str)

### Data Cleaning

In [ ]:
# as usual, we need to clean up the messy data
raw_text_data[:3000]

'\ufeffThe Project Gutenberg eBook of The Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: The Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease Date: September, 1997 [eBook #1041]\r\n[Most recently updated: December 3, 2022]\r\n\r\nLanguage: English\r\n\r\n\r\nProduced by:  the Project Gutenberg Shakespeare Team\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE SONNETS ***\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\nI\r\n\r\nFrom fairest creatures we desire increase,\r\nThat thereby 

In [ ]:
# Which characters could we use with the split() method to split the text into lines?

# split the text into **lines** and save the result to `split_data`
split_data = raw_text_data.split("\n")

# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# we need to drop all the boiler plate text (i.e. titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 
split_data = [x.replace("\r", "") for x in split_data]
split_data = [x.replace(",", "") for x in split_data]
split_data[:80] 

['From fairest creatures we desire increase',
 'That thereby beauty’s rose might never die',
 'But as the riper should by time decease',
 'His tender heir might bear his memory:',
 'But thou contracted to thine own bright eyes',
 'Feed’st thy light’s flame with self-substantial fuel',
 'Making a famine where abundance lies',
 'Thyself thy foe to thy sweet self too cruel:',
 'Thou that art now the world’s fresh ornament',
 'And only herald to the gaudy spring',
 'Within thine own bud buriest thy content',
 'And tender churl mak’st waste in niggarding:',
 '    Pity the world or else this glutton be',
 '    To eat the world’s due by the grave and thee.',
 '',
 'II',
 '',
 'When forty winters shall besiege thy brow',
 'And dig deep trenches in thy beauty’s field',
 'Thy youth’s proud livery so gazed on now',
 'Will be a tatter’d weed of small worth held:',
 'Then being asked where all thy beauty lies',
 'Where all the treasure of thy lusty days;',
 'To say within thine own deep sunken eyes

**Use list index slicing to remove the titles and descriptions, so we only have the sonnets.**


In [ ]:
# we need to drop all the boilerplate text (i.e., titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 

# find index boundaries (start, end)so that 
# sonnets exist between these indices 
# titles and descriptions exist outside of these indices

# use index slicing to isolate the sonnet lines from the text - save the result to `sonnets`s
sonnets = split_data[36:]
# YOUR CODE HERE
#raise NotImplementedError()

Notice that there are many lines that should not be counted as sonnets!

In [ ]:
# these non-sonnet lines have far fewer characters than the actual sonnet lines?

sonnets[200:240]

['',
 'XV',
 '',
 'When I consider everything that grows',
 'Holds in perfection but a little moment',
 'That this huge stage presenteth nought but shows',
 'Whereon the stars in secret influence comment;',
 'When I perceive that men as plants increase',
 'Cheered and checked even by the self-same sky',
 'Vaunt in their youthful sap at height decrease',
 'And wear their brave state out of memory;',
 'Then the conceit of this inconstant stay',
 'Sets you most rich in youth before my sight',
 'Where wasteful Time debateth with Decay',
 'To change your day of youth to sullied night',
 '    And all in war with Time for love of you',
 '    As he takes from you I engraft you new.',
 '',
 'XVI',
 '',
 'But wherefore do not you a mightier way',
 'Make war upon this bloody tyrant Time?',
 'And fortify yourself in your decay',
 'With means more blessed than my barren rhyme?',
 'Now stand you on the top of happy hours',
 'And many maiden gardens yet unset',
 'With virtuous wish would bear you liv

In [ ]:
# use your judgement to decide on a good value for  
#   the  minimum number of characters that a sonnet should have
#   call it n_chars
n_chars = 1000

# Let's use that observation to filter out all the non-sonnet lines!
#    save results to `filtered_sonnets`
# Hint: use a list comprehension
filtered_sonnets = []
tot_char = 0
for x in sonnets:
  tot_char = tot_char + len(x)
  if tot_char < n_chars:
    filtered_sonnets.append(x)



# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
filtered_sonnets = [s.lower().replace(";", "").replace(":","").replace(",","") for s in filtered_sonnets]
filtered_sonnets

['whose fresh repair if now thou not renewest',
 'thou dost beguile the world unbless some mother.',
 'for where is she so fair whose unear’d womb',
 'disdains the tillage of thy husbandry?',
 'or who is he so fond will be the tomb',
 'of his self-love to stop posterity?',
 'thou art thy mother’s glass and she in thee',
 'calls back the lovely april of her prime',
 'so thou through windows of thine age shalt see',
 'despite of wrinkles this thy golden time.',
 '    but if thou live remember’d not to be',
 '    die single and thine image dies with thee.',
 '',
 'iv',
 '',
 'unthrifty loveliness why dost thou spend',
 'upon thyself thy beauty’s legacy?',
 'nature’s bequest gives nothing but doth lend',
 'and being frank she lends to those are free',
 'then beauteous niggard why dost thou abuse',
 'the bounteous largess given thee to give?',
 'profitless usurer why dost thou use',
 'so great a sum of sums yet canst not live?',
 'for having traffic with thyself alone',
 'thou of thyself th

### Use Custom Data Cleaning Tool 

Use one of the methods in the `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [ ]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`
dctk = data_cleaning_toolkit()
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`
clean_sonnets = [dctk.clean_data(s) for s in filtered_sonnets]
# YOUR CODE HERE
#raise NotImplementedError()


In [ ]:
# much better!
display(clean_sonnets)
print(len(clean_sonnets))


['whose fresh repair if now thou not renewest',
 'thou dost beguile the world unbless some mother',
 'for where is she so fair whose uneard womb',
 'disdains the tillage of thy husbandry',
 'or who is he so fond will be the tomb',
 'of his selflove to stop posterity',
 'thou art thy mothers glass and she in thee',
 'calls back the lovely april of her prime',
 'so thou through windows of thine age shalt see',
 'despite of wrinkles this thy golden time',
 'but if thou live rememberd not to be',
 'die single and thine image dies with thee',
 '',
 'iv',
 '',
 'unthrifty loveliness why dost thou spend',
 'upon thyself thy beautys legacy',
 'natures bequest gives nothing but doth lend',
 'and being frank she lends to those are free',
 'then beauteous niggard why dost thou abuse',
 'the bounteous largess given thee to give',
 'profitless usurer why dost thou use',
 'so great a sum of sums yet canst not live',
 'for having traffic with thyself alone',
 'thou of thyself thy sweet self dost dece

27


### Use Your Data Tool to Create Character Sequences 
for the LSTM model

We'll need the `create_char_sequences` method for this task. <br>
However, this method requires a parameter called `maxlen,` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

To answer that question, let's do some statistics! 

In [ ]:
def calc_stats(corpus):
    """
    Calculates statistics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnet's line length - save the results to `doc_lens` 
    doc_lens = [len(x.split()) for x in clean_sonnets]
    # use numpy to calculate and return the mean, median, std, max, min of the doc lens - all in one line of code
    mean, median, std, max, min = np.mean(doc_lens), np.median(doc_lens), np.std(doc_lens), np.max(doc_lens), np.min(doc_lens)
    return (mean, median, std, max, min)
    # YOUR CODE HERE
    #raise NotImplementedError()


In [ ]:
# sonnet line length statistics 
mean, med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(6.777777777777778, 7.0, 2.629368792488718, 10, 0)

In [ ]:
# from the results of the sonnet line length statistics, use your judgement to select a value for maxlen
#   hint -- a good value might be half the median length of a sonnet line
# use .create_char_sequences() to create sequences
maxlen = 7
dctk.create_char_sequences(clean_sonnets)
# YOUR CODE HERE
#raise NotImplementedError()

Created 194 sequences.


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. <br>
Let's call them in the cells below.

In [ ]:
# number of input features for our LSTM model
dctk.n_features

24

In [ ]:
# unique characters that appear in our sonnets 
len(dctk.unique_chars)

24

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e., **dctk.unique_chars**) and the `number of model input features` (i.e., **dctk.n_features**) the same?
because that is the number of weights learned from the data.

**Hint:** The model that we will shortly build here is very similar to the text generation model we built in the guided project.

**Answer 1:**

because that is the number corresponding to the weights learned from the data.


**Question 2:**

Take a look at the printout of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

because we need to separate words.

----

### Use Our Data Tool to Create X and Y Splits

You'll need the `create_X_and_Y` method for this task. 

In [ ]:
# TODO: provide a walkthrough of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [ ]:
# notice that our input array isn't a matrix - it's a rank three tensor
X.shape

(194, 20, 24)

In $X$.shape, we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [ ]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False,

Notice that each sequence (i.e., $X[i]$ where $i$ is some index value) is `maxlen` long and <br>
has a number of features equal to `dctk.n_features`. <br>Let's try to understand this shape.

In [ ]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(20, 24)

**Each row corresponds to a character vector,** and there is `maxlen` number of character vectors. 

**Each column corresponds to a unique character,** and there are `dctk.n_features` number of features. 


In [ ]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

Notice that there is a single `True` value, and all the rest of the values are `False`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will be a single `True` value, and the rest will be `False`. 

Let's say that `True` appears in the $ith$ index; by  $ith$ index we mean some index in the general case. So how can we find out which character corresponds to?

To answer this question, we need to use the character-to-integer look-up dictionary. 

In [ ]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that character vector is endcoding for
dctk.int_char

{0: 'k',
 1: 'w',
 2: 'm',
 3: 'e',
 4: 'd',
 5: 'y',
 6: 'p',
 7: 'i',
 8: 'h',
 9: 's',
 10: 'r',
 11: 'c',
 12: 'u',
 13: 'o',
 14: 'b',
 15: 't',
 16: 'q',
 17: ' ',
 18: 'g',
 19: 'l',
 20: 'v',
 21: 'f',
 22: 'a',
 23: 'n'}

In [ ]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

w
h
o
s
e
 
f
r
e
s
h
 
r
e
p
a
i
r
 
i
Sequence length: 20


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape printout of `X.shape` to a classmate?


**Answer 1:**
It is the number of dimensions for the rows and columns

----


### Build a Shakespeare Sonnet Text Generation Model

Now that we have prepped our data (and understood that process), let's finally build out our character generation model, similar to what we did in the guided project.<br>

First, we'll create a callback to monitor the training -- by printing a sample of text generated by the model at the end of each epoch.

Helper function to generate a sample character:

In [ ]:
def sample(preds, temperature=1.0):
    """
    Helper function to generate a sample character
    Input is a predictions vector from our model, for example a set of 27 character probabilities
    Output is the index of the generated character 
    """
    # convert predictions to an array 
    preds = np.asarray(preds).astype('float64')

    # use the temperature hyper-parameter to "warp" (sharpen or spread out) the probability distribution 
    preds = np.log(preds) / temperature

    # use the softmax activation function to create a new list of probabilities 
    #   corresponding to the "warped" probability distribution
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # Draw a single sample from a multinomial distribution, given these probabilities
    #   The sample will be a one-hot encoded character
    """ Notes on the np.random.multinomial() function 
       The first argument is the number of "trials" we want: 1 in this case
       The second argument is the list of probabilities for each character
       The third argument is number of sets of "trials" we want: again, 1 in this case
       By analogy with a dice-rolling experiment: 
          This "trial" consists of generating a single "throw" of a die with 27 faces;
             each face corresponds to a character and its associated probability
    """

    probas = np.random.multinomial(1, preds, 1)
    
    # return the index that corresponds to the max probability 
    return np.argmax(probas)


Create the `on_epoch_end` function to be passed into `LambdaCallback()`

In [ ]:
def on_epoch_end(epoch, _):
    """"
    Function invoked at the end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunece into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next maxlen chars should be that follow the seed string
    for i in range(maxlen):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)
print(f'All of Shakespeare\'s sonnets comprise about {len(text)} characters')

All of Shakespeare's sonnets comprise about 987 characters


Create the callback object

In [ ]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Build and Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load (in the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [ ]:
# build text generation model layer by layer 
# fit model
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)

model = Sequential()

# hidden layer 1 
model.add(LSTM(128, 
               input_shape=(dctk.maxlen, dctk.n_features), # input_shape is (20,27)
               return_sequences=False)) # whenever using 2 or more LSTM layers, set return_sequences= True for all but the last LSTM layer

# this is our output layer
# recall that n_features = number of characters in the dictionary = 27
model.add(Dense(dctk.n_features, 
                activation='softmax'))

# notice that we are using categorical_crossentropy this time around - why?
model.compile(loss='categorical_crossentropy', 
              optimizer=opt)

# fit the model
# X and y are pretty large, consider sub-sampling
model.fit(X, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])
# YOUR CODE HERE
#raise NotImplementedError()

Epoch 1/50
1/2 [==============>...............] - ETA: 2s - loss: 3.1764
----- Generating text after Epoch: 0
----- Generating with seed: "hy golden time but i"
hy golden time but iyvoghyr
2/2 [==============================] - 3s 872ms/step - loss: 3.1744
Epoch 2/50
1/2 [==============>...............] - ETA: 0s - loss: 3.1517
----- Generating text after Epoch: 1
----- Generating with seed: "hyself alone thou of"
hyself alone thou ofholvdpb
2/2 [==============================] - 1s 421ms/step - loss: 3.1482
Epoch 3/50
1/2 [==============>...............] - ETA: 0s - loss: 3.1232
----- Generating text after Epoch: 2
----- Generating with seed: " then how when natur"
 then how when naturlnnn ki
2/2 [==============================] - 0s 413ms/step - loss: 3.1209
Epoch 4/50
1/2 [==============>...............] - ETA: 0s - loss: 3.0966
----- Generating text after Epoch: 3
----- Generating with seed: " thou abuse the boun"
 thou abuse the bounepaeiof
2/2 [==============================] - 0

### Save the trained model to a file

In [ ]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's Play With Our Trained Model 

Now that we have a trained model that, though far from perfect, can generate actual English words, we can look at the predictions to continue learning more about how a text generation model works.

We can also take this as an opportunity to unpack the `def on_epoch_end` function to understand better how it works. 

In [ ]:
# this is our joined clean sonnet data
text

'whose fresh repair if now thou not renewest thou dost beguile the world unbless some mother for where is she so fair whose uneard womb disdains the tillage of thy husbandry or who is he so fond will be the tomb of his selflove to stop posterity thou art thy mothers glass and she in thee calls back the lovely april of her prime so thou through windows of thine age shalt see despite of wrinkles this thy golden time but if thou live rememberd not to be die single and thine image dies with thee  iv  unthrifty loveliness why dost thou spend upon thyself thy beautys legacy natures bequest gives nothing but doth lend and being frank she lends to those are free then beauteous niggard why dost thou abuse the bounteous largess given thee to give profitless usurer why dost thou use so great a sum of sums yet canst not live for having traffic with thyself alone thou of thyself thy sweet self dost deceive then how when nature calls thee to be gone what acceptable audit canst thou leave'

In [ ]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

904

In [ ]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e., input sequence into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

In [ ]:
# display the "seed string" i.e. the input sequence into the model
print('----- Input seed: "' + sentence + '"')

----- Input seed: "e then how when natu"


In [ ]:
# use model to predict what the next maxlen chars should be that follow the seed string
for i in range(maxlen):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [ ]:
# this is the seed string
generated

'e then how when natu'

In [ ]:
# these are the maxlen chars that the model thinks should come after the seed stirng
sentence

'how when natuseesetr'

In [ ]:
# how put it all together
generated + sentence

'e then how when natuhow when natuseesetr'

# Resources and Stretch Goals

## Stretch Goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g., plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_generation) - code for training an RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem and provides an example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN